In [ ]:
import os
import numpy as np
import warnings
from astropy.coordinates import Angle
from astropy.io import fits

from astropy.modeling import models, fitting
import glob
from barycorrpy import get_BC_vel
from scipy import interpolate
import matplotlib.pyplot as plt
import json
import time
%matplotlib inline

In [ ]:
from AlgorithmDev import RadialVelocity

In [ ]:
def plot_gaussian_on_curve(g_curve, curve_x, curve_y, order=None, title=None, harps=None):
    plt.figure(figsize=(8,8))
    
    label_curve = "rv on order " + str(order) if order is not None else 'all orders '
    plt.plot(curve_x, curve_y, 'ko', label=label_curve)
    if harps is not None:
        plt.plot(curve_x, harps, 'mo', label="harps_ccf"  )
    plt.plot(curve_x, g_curve(curve_x), label='mean:'+str("{0:.6f}".format(g_curve.mean.value)))
    plt.legend(loc="lower right", prop={'size': 12})
    plt.title(title) if title is not None else None
    plt.show()

In [ ]:
#for f in range(total_file):
t0 = time.clock()
config = {
    'starname': 'tauceti',
    'star_rv': -16.5,
    'obslon': -70.7345,
    'obslat': -29.258,
    'obsalt': 2400.0, #in meters
    'ra2000': Angle("01:44:04.0915236842 hours").deg,
    'de2000': Angle("-15:56:14.934780748 degrees").deg,
    'pm_ra': -1729.726,  #mas/yr
    'pm_dec': 855.493, #mas/yr
    'parallax': 273.96, #mas
    'step': 0.25,
    'mask_width': 0.5 #km/s on either side of line center
    }


import pdb; pdb.set_trace()
spectrum_order = 70 
spectrum_path = '../test_data/order_trace_test/for_optimal_extraction/output/'
mask_path = '../test_data/rv_test/stellarmasks/G2.harps.mas'
outfolder = '../test_data/order_trace_test/for_radial_velocity/output'
#spectraname = glob.glob(spectrum_path + '/*.fits')
#spectraname.sort()
spectraname = [spectrum_path+'A_18_optimal_from_2.fits']
#wavecalib_file = '../test_data/order_trace_test/for_radial_velocity/thar_coadded_master_B.fits'
total_file = len(spectraname)

publicfile = '../test_data/rv_test/HARPStauceti_baseline/ccf/HARPS.2004-10-02T01_04_07.065_ccf_G2_A.fits'
harps_ccf, ccf_head = fits.getdata(publicfile, header=True)

radial_vel = RadialVelocity(config, spectrum_order, mask_path)
print(spectraname)

if radial_vel.init_calculation():
    for f in range(total_file):
        print('File ', f+1, 'of ', total_file)
    
        i1 = spectraname[f].index('output')+7
        i2 = spectraname[f].index('.fits')
        outfile = outfolder + '/rv_'+spectraname[f][i1:i2]+'.fits'

        
        o_spectrum, hdr = fits.getdata(spectraname[f], header=True)
        spectrum = o_spectrum * 2.0
        import pdb; pdb.set_trace()
        save_ccf = radial_vel.get_rv_on_spectrum(spectrum, hdr, harps_ccf, wavelength_calib_file=wavecalib_file)
        output_ccf = radial_vel.analyze_ccf(save_ccf)      #save_ccf is the same of output_ccf
        
        gaussian_fit, rv_result, g_x, g_y = radial_vel.fit_ccf(output_ccf)
        #harps_fit, harps_mean, h_x, h_y = radial_vel.fit_ccf(harps_ccf)

        plot_gaussian_on_curve(gaussian_fit, g_x, g_y, title=spectraname[f])
        #radial_vel.output_ccf_to_fits(output_ccf, outfile, ccf_head, rv_result)
        
t3 = time.clock()

In [ ]:
print('total time = ', (t3-t0), 'sec')

In [ ]:
import pdb;pdb.set_trace()
output_ccf = radial_vel.analyze_ccf(save_ccf)      #save_ccf is the same of output_ccf
gaussian_fit, rv_result, g_x, g_y = radial_vel.fit_ccf(output_ccf)